# Imports

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

# Hämta datan

In [48]:
iris = pd.read_csv("iris.csv")
mallcustomers = pd.read_csv("Mall_Customers.csv")

# Iris

In [49]:
# Vi kikar lite på iris datasetet
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [50]:
# Vi kikar efter saknande värden
missing_values = iris.isna().sum()
print("Missing values count: ", missing_values.sum())

Missing values count:  0


Inga värden saknas i alla fall.

In [56]:
sns.pairplot(iris)
plt.show()

NameError: name 'sns' is not defined

# Mall Customers

In [ ]:
# Vi kikar lite på mall customers datasetet
mallcustomers.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [ ]:
# Vi kan ta bort customer ID, den kolumnen har ingen betydelse för våran analys
mallcustomers = mallcustomers.iloc[:,1:]
mallcustomers.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


Ett annat problem är att gender är strängar just nu, vi behöver konvertera det till tal för att det ska fungera som en vektor.

In [ ]:
# Konvertera strängar för kön till tal
mallcustomers["Gender"] = mallcustomers["Gender"].map({"Male": 1, "Female": 2})
mallcustomers.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,1,21,15,81
2,2,20,16,6
3,2,23,16,77
4,2,31,17,40


# Min egna K-Means

In [ ]:
class MyKMeans:
    def __init__(self, k:int) -> None:
        self._data = None
        self._k = k

    def fit(self, data:pd.DataFrame) -> None:
        self._data = data

    def predict(self) -> str:
        if self._data is None:
            return

In [ ]:
MyKMeans(k=3)